In [110]:
import pandas as pd
import re


In [139]:
# Tool to delete unicode codes
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")
                 )
    return TEXT

#Reading CSV file
dataset=pd.read_csv('tweets3.csv', sep=',',quotechar='"',header=None,)
#Replacing Unicode characters
dataset[2]=dataset[2].apply(unicodetoascii)
#Deleting Urls
delete_urls = lambda x:re.sub(r'http\S+', '', x)
dataset[2] = dataset[2].apply(delete_urls)
#Deleting "
delete_quotes = lambda x:re.sub(r'"', '', x)
dataset[2] = dataset[2].apply(delete_quotes)
#Deleting b
delete_b = lambda x:re.sub(r'b\'', '', x)
dataset[2] = dataset[2].apply(delete_b)

#Datasets
X=dataset[2].values
Y=dataset[3].values
print(X,Y)

[ 'Our prayers go out to families of this tragic Christmas Eve plane crash in Bartow. We will remember your loss. @BN9... '
 "bOn @MSNBC w/ @AliVelshi discussing our @HispanicCaucus storming @SenSchumer's Office for #Dreamers #Sayfie "
 "Here's #GOPTaxScam chart. Rich get big tax cut. Middle Class get scraps, $2.3T in new debt &amp; cuts to SS &amp; Medicare... "
 ...,
 "Cosponsored the Protecting Internet Freedom Act to stop #Obama from giving away control of the #internet. @RepSeanDuffy #InternetGiveaway'"
 'Great to meet with #EmoryScholarsinDC this morning! @EmoryUniversity '
 'Voted yesterday to block cash ransom payments to Iran \\xe2\\x9e\\x99 '] [ 1  1  1 ..., -1 -1 -1]
